### MySQL database

Man kan koble opp mot forskjellige databaser i Python:
- GadFly
- mSQL
- **MySQL**
- PostgreSQL
- Microsoft SQL Server 2000
- Informix
- Interbase
- Oracle
- Sybase
- …

Eller bruke SqlLite3 
- Egen database modul for Python



In [1]:
# Viser hvordan vi kobler oss mot MySQL og leser/oppdaterer data.
# Må installere PyMySQL først: pip install pymysql
import pymysql

For å logge oss inn på databasen, må vi ha en konto på database-serveren.

Følgende informasjon må vi ha:

- host: navn på database-server
- user: brukernavn for å koble oss til serveren
- password: passord for å koble oss til serveren
- database: database seed (en database server kan inneholde mange database instanser)

Dersom vi ikke ønsker å "hardkode" passord (skrive det i programmet i klar tekst eller i en fil), kan vi bruke modulen **getPass** for å taste inn passordet når programmet starter.


In [ ]:
# importer funksjonen getpass fra modulen getpass.
# Vi bruker getpass for å lese passord fra bruker (slik at det ikke vises i klartekst)
from getpass import getpass

In [ ]:

# Definer variabler for oppkoblingsparameteret
my_host = "mysql.stud.iie.ntnu.no"
my_user = "rouhani" # Skriv inn brukernavnet ditt her
my_password = getpass()
my_database = "rouhani"

### Koble opp mot databasen og les fra en tabell

In [ ]:


with pymysql.connect(host = my_host, user = my_user, password = my_password, database = my_database) as my_db:
    my_cursor = my_db.cursor()

    my_sql = "SELECT regnr,navn,regdato,regav FROM gjenstand"
    my_cursor.execute(my_sql)

    for row in my_cursor:
        print(row)



### Koble opp mot databasen og oppdater rader i en tabell


### Koble opp mot databasen og fjern rader fra en tabell